In [1]:
from sympy import symbols, integrate, \
    cos, sin, pi, diff, Matrix, simplify, factor
from sympy.physics.vector import ReferenceFrame, curl, gradient
from sympy.vector import CoordSys3D

In [2]:
x = symbols("x")
integrate(cos(x)*sin(x), (x, 0, 2*pi))

0

In [3]:
integrate(sin(x)**3, (x, 0, pi))

4/3

### Potential and Conservative Vec Field

Test if $$ F =
\begin{bmatrix}
    x(z + 1) \\ y (z + 1) \\ -xyz
\end{bmatrix}
$$
Conservative? 


In [4]:
x, y, z = symbols("x y z")
F1 = x*(z + 1); F2 = y*(z + 1); F3 = -x*y*z;
R = ReferenceFrame("R")
Substitutions = [(x, R[0]), (y, R[1]), (z, R[2])]
F1_r = F1.subs(Substitutions)*R.x
F2_r = F2.subs(Substitutions)*R.y
F3_r = F3.subs(Substitutions)*R.z
F_r = F1_r + F2_r + F3_r
display(F_r)
curl(F_r, R)

R_x*(R_z + 1)*R.x + R_y*(R_z + 1)*R.y - R_x*R_y*R_z*R.z

(-R_x*R_z - R_y)*R.x + (R_x + R_y*R_z)*R.y

This shit is not conservative. 

Test if $$
\begin{bmatrix}
    y + z\cos(xz)
    \\
    x 
    \\
    x\cos(xz)
\end{bmatrix}
$$
is conservative, and then find the potential if it is. 

In [5]:
x, y, z = symbols("x y z")
F1 = y + z*cos(x*z); F2 = x; F3 = x*cos(x*z);
R = ReferenceFrame("R")
Substitutions = [(x, R[0]), (y, R[1]), (z, R[2])]
F1_r = F1.subs(Substitutions)*R.x
F2_r = F2.subs(Substitutions)*R.y
F3_r = F3.subs(Substitutions)*R.z
F_r = F1_r + F2_r + F3_r
display(F_r)
curl(F_r, R)


(R_y + R_z*cos(R_x*R_z))*R.x + R_x*R.y + R_x*cos(R_x*R_z)*R.z

0

Yep, this shit is conservative. Let's find the potential of the vector field: 

In [6]:
F1_Int = integrate(F1, x)
display(F1_Int)
F2_Int = integrate(F2, x)
display(F2_Int)
F3_Int = integrate(F3, x)
display(F3_Int)

x*y + z*Piecewise((sin(x*z)/z, Ne(z, 0)), (x, True))

x**2/2

Piecewise((x*sin(x*z)/z + cos(x*z)/z**2, Ne(z, 0)), (x**2/2, True))

Let's just add then up and define something.

In [7]:
Phi_x = x*y + sin(x*z)
Phi_y = x**2/2
Phi_z = (x*sin(z*x))/z + (cos(x*z))/z**2
Phi = Phi_x + Phi_y + Phi_z

diff(Phi, x)

x*cos(x*z) + x + y + z*cos(x*z)

Oh shit, so this is the wrong way to do it, we will have to be a bit more careful about this shit when integrating.

We should start with one of the function without any complications and then integrating backwards to find the answers.

Give the serface $z = x^2 + y^2$ find the scalar for the surface integrand of this surface.

In [8]:
x, y, z = symbols("x y z")
z = x**2 + y**2

Paramtrization of the surface

In [9]:
Surface = Matrix([[x], [y], [x**2 + y**2]])
Surf_Partialx = diff(Surface, x)
Surf_Partialy = diff(Surface, y)
display(Surf_Partialx)
display(Surf_Partialy)
Crossed = Surf_Partialx.cross(Surf_Partialy)
Crossed.norm()

Matrix([
[  1],
[  0],
[2*x]])

Matrix([
[  0],
[  1],
[2*y]])

sqrt(4*Abs(x)**2 + 4*Abs(y)**2 + 1)

Given the surface:
$
r(u, v) =
\begin{bmatrix}
    uv \\ v^2 \\ \frac{u^2}{v}
\end{bmatrix}
$
Find the surface integrator, scalr form.

In [10]:
u, v = symbols("u v")
r = Matrix([[u*v], [v**2], [u**2/2]])
r_Integrator = diff(r, u).cross(diff(r, v))
display(r_Integrator)
display(r_Integrator.norm())
factor(r_Integrator.norm()**2)

Matrix([
[-2*u*v],
[  u**2],
[2*v**2]])

sqrt(Abs(u**2)**2 + 4*Abs(v**2)**2 + 4*Abs(u*v)**2)

Abs(u**2)**2 + 4*Abs(v**2)**2 + 4*Abs(u*v)**2

### Stoke's Theorem
Given a vector field,
$
F = \begin{bmatrix}
    x \\ x + y \\ x + y + z
\end{bmatrix}
$, and then we are interested in the ellipse formed by the intersection of $x^2 + y^2 = 1$ and $z = y$, find the work done on the ellipse, counterclockwise.

Doing it old fasion with the Line integral.

In [11]:
x, y, z = symbols("x y z")
F = Matrix([[x], [x + y], [x + y + z]])
theta, r = symbols("theta r")
F = F.subs({x: r*cos(theta), y: r*sin(theta)})
display(F)

Matrix([
[                   r*cos(theta)],
[    r*sin(theta) + r*cos(theta)],
[r*sin(theta) + r*cos(theta) + z]])

The range for the integral is just the regular, $0\leq \theta \leq 2\pi$ and $r = 1$, and we also need to paramatrized the line

In [12]:
r  = Matrix([[cos(theta)], [sin(theta)], [sin(theta)]])
F  = Matrix([[x], [x + y], [x + y + z]])
dr = diff(r, theta)
display(dr)
F = F.subs([(x, r[0]), (y, r[1]), (z, r[2])])
display(F)
F_dot_dr = F.dot(dr)
display(F_dot_dr)
F_dot_dr = simplify(F_dot_dr)
display(F_dot_dr)
integrate(F_dot_dr, (theta, 0, 2*pi))

Matrix([
[-sin(theta)],
[ cos(theta)],
[ cos(theta)]])

Matrix([
[               cos(theta)],
[  sin(theta) + cos(theta)],
[2*sin(theta) + cos(theta)]])

(sin(theta) + cos(theta))*cos(theta) + (2*sin(theta) + cos(theta))*cos(theta) - sin(theta)*cos(theta)

sqrt(2)*sin(2*theta + pi/4) + 1

2*pi

The results for the line integral is $2\pi$, now we are going to use the idea of a surface integral and sum up the curl dot with the surface, and hopefully the deafult setting should get us safe with the right hand rule.

We also need to find out the curl of the vector field, but but in a different coordinate...?

### Alternative Curvillinear Coordiantes:

In [13]:
U = CoordSys3D("U", transformation="spherical")


